## CESM2 - LARGE ENSEMBLE (LENS2)

#### by Mauricio Rocha and Dr. Gustavo Marques

- The goal of this notebook is to calculate the MOC in density strings. We use https://github.com/sgyeager/POP_MOC as reference; thus, We thank Dr. Stephen Yeager. We also thank Michael Levy for the technical support. 

In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import copy
import scipy.stats
from scipy import signal
from functools import partial
import glob
import dask
import cf_xarray
import intake
import intake_esm
import matplotlib.pyplot as plt
from xhistogram.xarray import histogram
import pop_tools
%matplotlib inline
from MOCutils import popmoc
from dask.distributed import Client
from ncar_jobqueue import NCARCluster#,PBSCluster

## Improve the workflow using clusters 

In [2]:
#cluster = NCARCluster(memory='10 GB')
#cluster.scale(20)
mem_per_worker = 90 # in GB more memory here maybe 100 GB
num_workers = 45 # more workers maybe 45
cluster = NCARCluster(cores=1, processes=1, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=1:mem={mem_per_worker}GB')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

/glade/u/apps/opt/conda/envs/npl-2022b/lib/python3.8/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40557 instead
  warnings.warn(


<Client: 'tcp://10.12.206.51:36847' processes=0 threads=0, memory=0 B>


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mauricio/mrocha/proxy/40557/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mauricio/mrocha/proxy/40557/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.51:36847,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mauricio/mrocha/proxy/40557/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Read in OGCM history file & MOC template file

In [3]:
fmoc = '/glade/u/home/yeager/analysis/python/POP_MOC/moc_template.nc'
ds_moctemp = xr.open_dataset(fmoc) # MOC template

In [4]:
# Open original collection description file
cat_url='/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
col = intake.open_esm_datastore(cat_url)

/glade/u/home/mauricio/.local/lib/python3.8/site-packages/intake_esm/utils.py:96: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(catalog_path, **csv_kwargs), catalog_path


In [5]:
print("Catalog file:", col.esmcol_data["catalog_file"])
col.df.head(10)

Catalog file: glade-cesm2-le.csv.gz


,component,stream,case,member_id,variable,start_time,end_time,time_range,long_name,units,vertical_levels,frequency,path,experiment,forcing_variant,cesm_member_id,control_branch_year,cmip_experiment_id
0,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1850-01-01,1859-12-31,18500101-18591231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
1,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1860-01-01,1869-12-31,18600101-18691231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
2,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1870-01-01,1879-12-31,18700101-18791231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
3,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1880-01-01,1889-12-31,18800101-18891231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
4,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1890-01-01,1899-12-31,18900101-18991231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
5,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1900-01-01,1909-12-31,19000101-19091231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
6,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1910-01-01,1919-12-31,19100101-19191231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
7,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1920-01-01,1929-12-31,19200101-19291231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
8,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1930-01-01,1939-12-31,19300101-19391231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
9,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1940-01-01,1949-12-31,19400101-19491231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1


In [6]:
import pprint
uniques_orig = col.unique(columns=["component", "frequency", "experiment", "variable"])
pprint.pprint(uniques_orig, compact=True, indent=1, width=80)

{'component': {'count': 6,
               'values': ['rof', 'ocn', 'lnd', 'glc', 'ice', 'atm']},
 'experiment': {'count': 2, 'values': ['ssp370', 'historical']},
 'frequency': {'count': 7,
               'values': ['hour_6', 'day_1', 'hour_3', 'year_1', 'day_365',
                          'month_1', 'hour_1']},
 'variable': {'count': 1906,
              'values': ['GROSS_NMIN', 'DSTDEP', 'DIA_IMPVF_DIC', 'TMCO2_LND',
                         'Lig_photochem', 'meltl_d', 'num_c1DDF',
                         'QRUNOFF_TO_COUPLER', 'KVMIX', 'daidtt_d', 'FSAT',
                         'QFLX', 'Jint_100m_ALK_ALT_CO2', 'TBRY', 'so4_a1SFWET',
                         'spP', 'CH4_SURF_AERE_UNSAT', 'tstep_count',
                         'num_a4_SRF', 'TLAI', 'TAUWMODIS', 'H2SO4_sfnnuc1',
                         'SOAG_SRF', 'TOTCOLN', 'HUMIDEX_U',
                         'photoC_sp_zint_100m', 'TMso4_a2', 'O2', 'alvdf_ai',
                         'VPWP_CLUBB', 'soa_a1SFWET', 'zoo_loss_doc_z

In [7]:
# Some variables like temperature, salinity, are not available for annual frequency, so we selected monthly. 
col.search(component="ocn", variable=["TEMP","SALT","UVEL","VVEL"], frequency="month_1", experiment="historical").df

,component,stream,case,member_id,variable,start_time,end_time,time_range,long_name,units,vertical_levels,frequency,path,experiment,forcing_variant,cesm_member_id,control_branch_year,cmip_experiment_id
0,ocn,pop.h,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,SALT,1850-01,1859-12,185001-185912,Salinity,gram/kilogram,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/o...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
1,ocn,pop.h,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,SALT,1860-01,1869-12,186001-186912,Salinity,gram/kilogram,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/o...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
2,ocn,pop.h,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,SALT,1870-01,1879-12,187001-187912,Salinity,gram/kilogram,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/o...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
3,ocn,pop.h,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,SALT,1880-01,1889-12,188001-188912,Salinity,gram/kilogram,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/o...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
4,ocn,pop.h,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,SALT,1890-01,1899-12,189001-189912,Salinity,gram/kilogram,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/o...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6795,ocn,pop.h,b.e21.BHISTsmbb.f09_g17.LE2-1301.020,r20i1301p1f2,VVEL,1970-01,1979-12,197001-197912,Velocity in grid-y direction,centimeter/s,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/o...,historical,smbb,1301.020,1301,CESM2_historical_r20i1301p1f2
6796,ocn,pop.h,b.e21.BHISTsmbb.f09_g17.LE2-1301.020,r20i1301p1f2,VVEL,1980-01,1989-12,198001-198912,Velocity in grid-y direction,centimeter/s,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/o...,historical,smbb,1301.020,1301,CESM2_historical_r20i1301p1f2
6797,ocn,pop.h,b.e21.BHISTsmbb.f09_g17.LE2-1301.020,r20i1301p1f2,VVEL,1990-01,1999-12,199001-199912,Velocity in grid-y direction,centimeter/s,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/o...,historical,smbb,1301.020,1301,CESM2_historical_r20i1301p1f2
6798,ocn,pop.h,b.e21.BHISTsmbb.f09_g17.LE2-1301.020,r20i1301p1f2,VVEL,2000-01,2009-12,200001-200912,Velocity in grid-y direction,centimeter/s,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/o...,historical,smbb,1301.020,1301,CESM2_historical_r20i1301p1f2


In [8]:
%%time
cat_subset = col.search(component='ocn',
                            variable=['TEMP','SALT','UVEL','VVEL'],
                            frequency='month_1',
                            experiment='historical',
                            forcing_variant='smbb',                               
                       )
dset_dict_raw = cat_subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
print(f"\nDataset dictionary keys:\n {dset_dict_raw.keys()}")


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



Dataset dictionary keys:
 dict_keys(['ocn.historical.pop.h.smbb.VVEL', 'ocn.historical.pop.h.smbb.UVEL', 'ocn.historical.pop.h.smbb.SALT', 'ocn.historical.pop.h.smbb.TEMP'])
CPU times: user 50.4 s, sys: 1.89 s, total: 52.3 s
Wall time: 1min 6s


### Compute sigma-2 field from LENS2 dataset

In [9]:
tslice = slice("1850-01-01", "2014-12-31") # selecting the period

In [10]:
%%time
# Salinity
ds_smbb_salt = dset_dict_raw['ocn.historical.pop.h.smbb.SALT'] # salinity
print(f"Salinty before: {dask.utils.format_bytes(ds_smbb_salt.nbytes)}")
ds_smbb_salt = ds_smbb_salt.sel(time=tslice)
ds_smbb_salt = ds_smbb_salt.resample(time='1Y', closed='left').mean('time') # Yearly average
salt = ds_smbb_salt['SALT']
salt = salt.mean(dim=["member_id"]) # Average of all members
print(f"Salinty after: {dask.utils.format_bytes(salt.nbytes)}")
salt = salt.load() # necessary because pop-tools.eos() doesn't play nicely with dask
del ds_smbb_salt

Salinty before: 2.66 TiB
Salinty after: 4.53 GiB


KilledWorker: ("('concatenate-e7e2bc77f8bc7735a1b976e374e8b52b', 0, 0, 0, 0)", <WorkerState 'tcp://10.12.206.28:40413', name: PBSCluster-44, status: closed, memory: 0, processing: 52>)

In [ ]:
%%time
# Temperature
ds_smbb_temp = dset_dict_raw['ocn.historical.pop.h.smbb.TEMP'] # temperature
print(f"Temperature before: {dask.utils.format_bytes(ds_smbb_temp.nbytes)}")
ds_smbb_temp = ds_smbb_temp.sel(time=tslice)
ds_smbb_temp = ds_smbb_temp.resample(time='1Y', closed='left').mean('time') # Yearly average
temp = ds_smbb_temp['TEMP']
temp = temp.mean(dim=["member_id"]) # Average of all members
print(f"Temperature after: {dask.utils.format_bytes(temp.nbytes)}")
temp = temp.load() # necessary because pop-tools.eos() doesn't play nicely with dask

In [ ]:
%%time
# Zonal velocity 
ds_smbb_uvel = dset_dict_raw['ocn.historical.pop.h.smbb.UVEL'] #  zonal velocity 
print(f"Zonal Velocity before: {dask.utils.format_bytes(ds_smbb_uvel.nbytes)}")
ds_smbb_uvel = ds_smbb_uvel.sel(time=tslice)
ds_smbb_uvel = ds_smbb_uvel.resample(time='1Y', closed='left').mean('time') # Yearly average
uvel = ds_smbb_uvel['UVEL']
uvel = uvel.mean(dim=["member_id"]) # Average of all members
print(f"Zonal Velocity before: {dask.utils.format_bytes(uvel.nbytes)}")
uvel = uvel.load() # necessary because pop-tools.eos() doesn't play nicely with dask

In [ ]:
%%time
# Meridional velocity 
ds_smbb_vvel = dset_dict_raw['ocn.historical.pop.h.smbb.VVEL'] #  meridional velocity 
print(f"Meridional Velocity before: {dask.utils.format_bytes(ds_smbb_vvel.nbytes)}")
ds_smbb_vvel = ds_smbb_vvel.sel(time=tslice)
ds_smbb_vvel = ds_smbb_vvel.resample(time='1Y', closed='left').mean('time') # Yearly average
vvel = ds_smbb_vvel['VVEL']
vvel = vvel.mean(dim=["member_id"]) # Average of all members
print(f"Zonal Velocity before: {dask.utils.format_bytes(vvel.nbytes)}")
vvel = vvel.load() # necessary because pop-tools.eos() doesn't play nicely with dask

In [ ]:
# Define k-index array
dims = np.shape(temp)
#ne = dims[0] # ensember member
nt = dims[0]  # time 
nz = dims[1]  # depth
ny = dims[2]  # latitude
nx = dims[3]  # longitude
kji = np.indices((nz,ny,nx))
kindices = kji[0,:,:,:] + 1 # (time, nlat, nlon)

In [ ]:
refz = 2000 # reference depth
refdep = xr.full_like(salt,refz).rename('REFDEP')
# Sigma2 on model TLAT, TLONG
sigma2_T = pop_tools.eos(salt=salt,temp=temp,depth=refdep) - 1000
sigma2_T = sigma2_T.assign_attrs({'long_name':'Sigma referenced to {}m'.format(refz),'units':'kg/m^3'})
sigma2_T = sigma2_T.mean(dim=["time"]) # Average over time
# apply T-grid mask
#mask=kindices<=ds['KMT'].values[None,:,:]
#sigma2_T = sigma2_T.where(mask)

### Define target sigma-2 vertical grid

In [ ]:
# Use predefined 86-layer sigma2 grid:
sigma_mid,sigma_edge = popmoc.sigma2_grid_86L()

### Compute MOC(Sigma2) using xhistogram 

#### 1. Compute Isopycnal Layer Thickness

In [ ]:
# Here, test histogram by counting cells in each density bin. Vertical sum should be same as KMT.
iso_count = histogram(sigma2_T, bins=[sigma_edge.values],dim=['z_t'],density=False)
iso_count = iso_count.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
kmtdiff = (iso_count.sum('sigma') - ds_smbb_temp['KMT'].mean(dim=["time"]))
print("Max difference from true KMT = {}".format(abs(kmtdiff).max().values))

In [ ]:
# Use histogram to compute layer thickness. Vertical sum should be same as HT.
dzwgts = (ds_smbb_temp['dz']/100.).assign_attrs({'units':'m'})
dzwgts = dzwgts.mean(dim=["time"]) # Average over time
iso_thick = histogram(sigma2_T, bins=[sigma_edge.values], weights=dzwgts,dim=['z_t'],density=False)
iso_thick = iso_thick.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
iso_thick = iso_thick.rename('Isopycnal Layer Thickness').assign_attrs({'units':'m'})
htdiff = iso_thick.sum('sigma') - (ds_smbb_temp['HT']/100.).assign_attrs({'units':'m'})
htdiff = htdiff.mean(dim=["time"]) # Average over time
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))
#In the original Notebook, the maximum difference is: HT = 1.2270752449694555e-05m

#### 2. Compute Isopycnal Layer Depth

In [ ]:
# Cumulative sum of layer thickness yields depth of layer edges:
iso_depth = iso_thick.cumsum('sigma').rename('Isopycnal Layer Depth')
iso_depth['sigma'] = sigma_edge.isel(sigma=slice(1,None))

In [ ]:
iso_depth.isel(sigma=84).plot(size=6,vmax=5500)

In [ ]:
# Isopycnal depth of bottom edge should be same as HT.
htdiff =  iso_depth.isel(sigma=-1) - (ds_smbb_temp['HT']/100.).assign_attrs({'units':'m'})
htdiff = htdiff.mean(dim=["time"]) # Average over time
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))
#Max difference from true HT = 1.2270752449694555e-05m

#### 3. Compute Isopycnal Layer Horizontal Volume Flux

In [ ]:
# Grid-oriented Volume FLuxes:
uvel = uvel.where(uvel<1.e30).fillna(0.)
vvel = vvel.where(vvel<1.e30).fillna(0.)
uvel = (uvel*ds_smbb_uvel['DYU']*ds_smbb_uvel['dz']/1.e6).assign_attrs({'units':'m^3/s'})
vvel = (vvel*ds_smbb_vvel['DXU']*ds_smbb_vvel['dz']/1.e6).assign_attrs({'units':'m^3/s'})
uvel = uvel.mean(dim=["time"]) # Average over time
vvel = vvel.mean(dim=["time"]) # Average over time

In [ ]:
# Volume fluxes in density-space. Vertical sum is density-space should reproduce vertical sum in depth-space.
iso_uflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=uvel,dim=['z_t'],density=False) # The 'numpy.histogram_bin_edges' function is not implemented by Dask array.
iso_uflux = iso_uflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
iso_vflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=vvel,dim=['z_t'],density=False)
iso_vflux = iso_vflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

ufluxdiff = iso_uflux.sum('sigma') - uvel.sum('z_t')
vfluxdiff = iso_vflux.sum('sigma') - vvel.sum('z_t')
print("Max difference from true Uflux = {}".format(abs(ufluxdiff).max().values))
print("Max difference from true Vflux = {}".format(abs(vfluxdiff).max().values))
#Max difference from true Uflux = 1367813.3765927013
#Max difference from true Vflux = 456888.7641561439

Need to investigate these differences, which appear to be associated with overflows. The difference plot below shows zero almost everywhere except near Nordic Seas overflow points.

In [ ]:
ufluxdiff.plot(size=7,vmin=-1.e5,vmax=1.e5)

#### 4. Compute Vertical Volume Flux using model divergence operator

In [ ]:
wflux = popmoc.pop_isowflux(iso_uflux,iso_vflux,'sigma',sigma_edge)

#### 5. Compute Zonal Sums of Vertical Volume Flux in latitude strips

In [ ]:
# Load predefined 1-degree target latitude grid:
lat_mid,lat_edge = popmoc.latitude_grid_1deg()

In [ ]:
## Define MOC region mask with legend:
rmask = ds_smbb_temp.REGION_MASK
rmask=rmask.mean(dim=["time"])
rmaskmoc = rmask.where(rmask>0)
rmaskmoc = xr.where((rmask>0),1,rmaskmoc)
rmaskmoc = xr.where((rmask>=6) & (rmask<=11),2,rmaskmoc)
rmaskmoc.plot(levels=[0,1,2,3]);
rmaskmoc.attrs['legend'] = {0:"Global",1:"IndoPac+SO",2:"Atlantic"}

In [ ]:
tarea = ds_smbb_temp['TAREA']
tarea=tarea.mean(dim=["time"])
tlat = ds_smbb_temp['TLAT']
wflux_zonsum = popmoc.mesh_zonalavg(wflux,tarea,tlat,rmaskmoc,rmaskmoc.legend,lat_edge,sum=True)

#### 6. Compute cumulative meridional integral of zonally-summed wflux

A southward cumulative integral from 90N avoids issues associated with southern boundary of Atlantic region.

In [ ]:
moc = -wflux_zonsum.sel(lat=slice(None,None,-1)).cumsum('lat').sel(lat=slice(None,None,-1))
moc = (moc/1.e6).assign_attrs({'units':'Sv'})   
moc.name = 'MOC'

In [ ]:
moc.isel(region=0).plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])

In [ ]:
moc.isel(region=1).plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])

In [ ]:
moc.isel(region=2).plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])

In [ ]:
moc.isel(region=[1,2]).sum('region').plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])